In [1]:
import numpy as np
import cvxpy as cp 
import matplotlib.pyplot as plt
from scipy.special import logsumexp, softmax
%matplotlib inline
%config InlineBackend.figure_format='retina'
from IPython.display import clear_output
import time
clear_output()

In [2]:
"""
data 
"""

np.random.seed(23824)

TOLDIV = 5e-10
MIT = 2.5e3

d = int(1e3)
fade_cand = 0
fade = max(-10,min(1,fade_cand))
p = 1
R_def = 10
temp = 1e-2

N_LSE = int(5e4)
LV,UV = -5,5

normalization = False

In [3]:
# from INRIA 
def proj_simplex_(vec, R=1,tol=1e-3):
    """ 
    References
    ----------
    [1] Efficient Projections onto the .1-Ball for Learning in High Dimensions
        John Duchi, Shai Shalev-Shwartz, Yoram Singer, and Tushar Chandra.
        International Conference on Machine Learning (ICML 2008)
        http://www.cs.berkeley.edu/~jduchi/projects/DuchiSiShCh08.pdf
    """
    assert R > 0, "radius R must be strictly positive (%d <= 0)" % R
    d, = vec.shape  # will raise ValueError if vec is not 1-D
    if abs(vec.sum() - R)<=tol and np.alltrue(vec >= 0):
        return vec
    sorted_vec = np.sort(vec)[::-1]
    cssv = np.cumsum(sorted_vec)
    buf = np.nonzero(sorted_vec * np.arange(1, d+1) > (cssv - R))[0][-1]
    shift = float(cssv[buf] - R) / (1+buf)
    w = (vec - shift).clip(min=0)
    return w

def proj_ball_(x,xi_embd=np.zeros(d),nm_power=p,R=R_def,tol=5e-4):
    dxxi_embd = x-xi_embd
    abs_dxxi_embd = np.abs(dxxi_embd)
    if nm_power==1:
        if np.sum(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + proj_simplex_(abs_dxxi_embd,R)*np.sign(dxxi_embd)
    elif nm_power==np.inf:
        if max(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + np.minimum(R,np.maximum(-R,dxxi_embd))
    elif nm_power==2:
        buf = (np.sum(abs_dxxi_embd**2))**(1/2)
        if buf<=R+tol:
            return x
        return xi_embd + dxxi_embd*R/max(R,buf)
    else:
        return None
    
k_inner_max_def = int(5e4)
def prox_bundle_update(G,ftilde,z,L,verb=False,k_inner_max=k_inner_max_def):
    local_bs = len(G)
    if local_bs==0:
        return proj_ball_(z),None
    elif local_bs==1:
        return proj_ball_(z-G[0]/L),np.array([1])
    else:
        weights = np.zeros(local_bs)
        weights[-1] = 1
        slack = 1
        k_inner = 0
        u_ = proj_ball_(z-weights@G/L)
        slack = max(G@u_+ftilde)-((weights@G)@u_ + weights@ftilde)
        while slack>1e-4 and k_inner<k_inner_max:
            grad_ = ftilde+G@u_
            changeID = np.argmax(grad_)
            weights =  k_inner/(k_inner+2)*weights
            weights[changeID]+=2/(k_inner+2)
            u_ = proj_ball_(z-weights@G/L)
            slack = max(G@u_+ftilde)-((weights@G)@u_ + weights@ftilde)
            k_inner += 1
        if slack>1e-4:
            print('inexact prox | slack = '+str(slack))
        return u_,weights

"""

active components

"""

def selection(x_new,G1,f1tilde,tol_act=5e-5):
    vals = np.array(G1)@x_new+np.array(f1tilde)
    buf = np.where(vals>=np.max(vals)-tol_act)[0]
    if len(buf)>0:
        G1_buf,f1tilde_buf = [],[]
        for elem in buf:
            G1_buf.append(G1[int(elem)])
            f1tilde_buf.append(f1tilde[int(elem)])
        return G1_buf,f1tilde_buf
    else:
        return [],[]

In [4]:
# from deepseek

def split_matrix(U):
    # Compute the eigenvalues and eigenvectors of U
    eigvals, eigvecs = np.linalg.eigh(U)
    
    # Split the eigenvalues into positive and negative parts
    lambda_plus = np.maximum(eigvals, 0)
    lambda_minus = np.maximum(-eigvals, 0)
    
    # Construct the diagonal matrices for positive and negative parts
    Lambda_plus = np.diag(lambda_plus)
    Lambda_minus = np.diag(lambda_minus)
    
    # Reconstruct U1 and U2 using the eigenvectors
    U1 = eigvecs @ Lambda_plus @ eigvecs.T
    U2 = eigvecs @ Lambda_minus @ eigvecs.T
    
    return (U1+U1.T)/2, (U2+U2.T)/2

In [5]:
def SDCe(MO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    tau_ext = .001
    x_stability_prev_next,x_stability_prev,x_stability,x,L = x_init.copy(),x_init.copy(),x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    th,th_prev,varsigma = 1,0,0
    
    DCA_useful_iters = 0

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        y = x_stability+min(np.sqrt(L_prev*(1-tau_ext)/L),varsigma)*(x_stability_prev_next-x_stability_prev)
        f1y,g1y = f1(y,mode='full')
        f1y_tilde = f1y-np.sum(g1y*y)
        if cond:
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,x,DCA_inner = MO(y,g1y,f1y_tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val + L*np.sum((y-x_stability)**2)/2 - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability_prev = x_stability
            x_stability_prev_next = x_stability_plus
            x_stability = x.copy()
            val = val_new
            new_th = (1+np.sqrt(1+4*th**2))/2
            th_prev = th
            th = new_th
            varsigma = (th_prev-1)/th

        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [6]:
def ASDCe(BMO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,filter_out=True,bs_max=max(min(d+5,1000),100),max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    tau_ext = .001
    x_stability_prev_next,x_stability_prev,x_stability,x,L = x_init.copy(),x_init.copy(),x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    th,th_prev,varsigma = 1,0,0
    
    DCA_useful_iters = 0
    
    G1 = []
    f1tilde = []

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        y = x_stability+min(np.sqrt(L_prev*(1-tau_ext)/L),varsigma)*(x_stability_prev_next-x_stability_prev)
        f1y,g1y = f1(y,mode='full')
        f1y_tilde = f1y-np.sum(g1y*y)
        if cond:
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)
            
        G1 = G1[-(bs_max-1):]
        G1.append(g1y)
        f1tilde = f1tilde[-(bs_max-1):]
        f1tilde.append(f1y_tilde)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        B_size = len(G1)
        x_stability_plus,weights,x,DCA_inner = BMO(y,G1,f1tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val + L*np.sum((y-x_stability)**2)/2 - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability_prev = x_stability
            x_stability_prev_next = x_stability_plus
            x_stability = x.copy()
            val = val_new
            new_th = (1+np.sqrt(1+4*th**2))/2
            th_prev = th
            th = new_th
            varsigma = (th_prev-1)/th
            
            # bundle management
            if filter_out:
                G1,f1tilde = selection(x_stability,np.array(G1),f1tilde)
                
        else:
            
            # bundle management
            aggregate_g = weights@G1
            aggregate_ftilde = weights@f1tilde
            ## getting rid of stability center info that will be reintroduced hereafter ##
            G1 = G1[:-1]
            f1tilde = f1tilde[:-1] 
            G1.append(aggregate_g)
            f1tilde.append(aggregate_ftilde)
            G1.append(g1x)
            f1tilde.append(f1x-np.sum(g1x*x))

        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters)+' ; B_size = '+str(B_size))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [7]:
def SDC(MO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    x_stability,x,L = x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    
    DCA_useful_iters = 0

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        if cond:
            y = x_stability.copy()
            f1y,g1y = f1(y,mode='full')
            f1y_tilde = f1y-np.sum(g1y*y)
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,x,DCA_inner = MO(y,g1y,f1y_tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability = x.copy()
            val = val_new
        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [8]:
def ASDC(BMO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,filter_out=True,bs_max=max(min(d+5,1000),100),max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    x_stability,x,L = x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    
    DCA_useful_iters = 0
    
    G1 = []
    f1tilde = []

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        if cond:
            y = x_stability.copy()
            f1y,g1y = f1(y,mode='full')
            f1y_tilde = f1y-np.sum(g1y*y)
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)
            
        G1 = G1[-(bs_max-1):]
        G1.append(g1y)
        f1tilde = f1tilde[-(bs_max-1):]
        f1tilde.append(f1y_tilde)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,weights,x,DCA_inner = BMO(y,G1,f1tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability = x.copy()
            val = val_new
            
            # bundle management
            if filter_out:
                G1,f1tilde = selection(x_stability,np.array(G1),f1tilde)
                
        else:
            
            # bundle management
            aggregate_g = weights@G1
            aggregate_ftilde = weights@f1tilde
            ## getting rid of stability center info that will be reintroduced hereafter ##
            G1 = G1[:-1]
            f1tilde = f1tilde[:-1] 
            G1.append(aggregate_g)
            f1tilde.append(aggregate_ftilde)
            G1.append(g1x)
            f1tilde.append(f1x-np.sum(g1x*x))
        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [9]:
import pandas as pd

In [10]:
NPROBS = int(5)
REPS = int(5)
TL = 2/3

times = []
values = []
repetitions = []
extraps = []
method = []
num_iters = []
DC_updates = []
ls = []
probIDs = []
dca_useful_iters = []

for probID in range(NPROBS):
    

    if normalization:
        betas = np.random.randint(LV,UV+1,(N_LSE,d))/max(UV,LV)
    else:
        betas = np.random.randint(LV,UV+1,(N_LSE,d))

    U = np.random.uniform(-2,2,(d,d))

    _, U2 = split_matrix(U)
    U2 = (1-fade)*U2
    U2 = (U2+U2.T)/2
    L_ref = 1000.0
    print(' ')
    print('PID #'+str(probID))
    print(' ')

    """
    information oracles
    """


    def f1_oracle(x,mode='value'):
        buf = (betas@x)/temp
        if mode=='value':
            return temp*logsumexp(buf)
        elif mode=='gradient':
            return softmax(buf)@betas
        else:
            return temp*logsumexp(buf),softmax(buf)@betas

    def f2_oracle(x,mode='value'):
        U2x = U2@x
        if mode=='value':
            return (1/2)*np.sum(U2x*x)
        elif mode=='gradient':
            return U2x
        else:
            return (1/2)*np.sum(U2x*x),U2x

    def Psi_oracle(x):
        if np.linalg.norm(x,p)<=R_def+3e-3:
            return 0
        else:
            return np.inf

    def F_oracle(x):
        return f1_oracle(x)-f2_oracle(x)+Psi_oracle(x)
    
    """
    oracle 

    approx solves: (subProblem) min mod(u) := f1(y) + <grad f1(y),u-y> + L_hat/2*||u-y||_2^2 + Psi(u) - f2(u)

    ensures with (x)_+ = arg min f1(y) + <grad f1(y),u-y> + L_hat/2*||u-y||_2^2 + Psi(u) - f2(x)-<g2(x),u-x>
    that

    output x_+ is a (approx) critical point of (subProblem) (i)
    output x_+ such that mod(x_+) <= mod((x)_+) (ii)


    """

    def MO_oracle(y,g1y,f1y_tilde,g2x,f2x_tilde,L_hat,max_steps=0,min_decr_inner=5e-4):
        mod = lambda u: f1y_tilde+np.sum(g1y*y)-f2_oracle(u,'value')+Psi_oracle(u)+L_hat*np.sum((u-y)**2)/2
        x_plus = proj_ball_(y-(g1y-g2x)/L_hat)
        x_ = x_plus.copy()
        modval = mod(x_)
        k_inner,cond = 0,True
        while k_inner<max_steps and cond:
            g2x_ = f2_oracle(x_,'gradient')
            x_next = proj_ball_(y-(g1y-g2x_)/L_hat)
            modval_next = mod(x_next)
            decr = modval-modval_next
            if decr>min_decr_inner:
                k_inner += 1
                x_ = x_next.copy()
                modval = modval_next
            else:
                cond = False
                if modval_next<modval:
                    x_ = x_next.copy()
        return x_plus,x_,k_inner
    
    
    def BMO_oracle(y,G1,f1tilde,g2x,f2x_tilde,L_hat,max_steps=0,min_decr_inner=5e-4):
        mod = lambda u: np.max(f1tilde+G1@y)-f2_oracle(u,'value')+Psi_oracle(u)+L_hat*np.sum((u-y)**2)/2
        x_plus,weights = prox_bundle_update(np.array(G1)-np.outer(np.ones(len(G1)),g2x),np.array(f1tilde)-f2x_tilde,z=y,L=L_hat)
        x_ = x_plus.copy()
        modval = mod(x_)
        k_inner,cond = 0,True
        while k_inner<max_steps and cond:
            f2x_,g2x_ = f2_oracle(x_,'full')
            x_next,weights_next = prox_bundle_update(np.array(G1)-np.outer(np.ones(len(G1)),g2x_),np.array(f1tilde)-(f2x_-np.sum(g2x_*x_)),z=y,L=L_hat)
            modval_next = mod(x_next)
            decr = modval-modval_next
            if decr>min_decr_inner:
                k_inner += 1
                x_ = x_next.copy()
                weights = weights_next.copy()
                modval = modval_next
            else:
                cond = False
                if modval_next<modval:
                    x_ = x_next.copy()
                    weights = weights_next.copy()
        return x_plus,weights,x_,k_inner

    for rep in range(REPS):

        x_random = proj_ball_(np.random.uniform(-3,3,d))

        print('repID: '+str(rep))
        print(' ')
        
        
        print(' ')

        print('go ASDCe')
        time_start_ASDCe = time.time()
        x_out_ASDCe,F_out_ASDCe,k_out_ASDCe,dca_used_ASDCe = ASDCe(BMO=BMO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=1)

        time_end_ASDCe = time.time()

        repetitions.append(rep)
        times.append(time_end_ASDCe-time_start_ASDCe)
        values.append(F_out_ASDCe)
        method.append('ASDCe')
        num_iters.append(k_out_ASDCe)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_ASDCe)
        

        print(' ')

        print('go SDCe-LS')
        time_start_SDCe_LS = time.time()
        x_out_SDCe_LS,F_out_SDCe_LS,k_out_SDCe_LS,dca_used_SDCe_LS = SDCe(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SDCe_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SDCe_LS-time_start_SDCe_LS)
        values.append(F_out_SDCe_LS)
        method.append('SDCe-LS')
        num_iters.append(k_out_SDCe_LS)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SDCe_LS)


        print(' ')

        print('go SCe-LS')
        time_start_SCe_LS = time.time()
        x_out_SCe_LS,F_out_SCe_LS,k_out_SCe_LS,dca_used_SCe_LS = SDCe(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(0),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SCe_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SCe_LS-time_start_SCe_LS)
        values.append(F_out_SCe_LS)
        method.append('SCe-LS')
        num_iters.append(k_out_SCe_LS)
        extraps.append('yes')
        DC_updates.append('no')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SCe_LS)
        
        
        print(' ')

        print('go ASDC')
        time_start_ASDC = time.time()
        x_out_ASDC,F_out_ASDC,k_out_ASDC,dca_used_ASDC = ASDC(BMO=BMO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_ASDC = time.time()

        repetitions.append(rep)
        times.append(time_end_ASDC-time_start_ASDC)
        values.append(F_out_ASDC)
        method.append('ASDC')
        num_iters.append(k_out_ASDC)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_ASDC)


        print(' ')


        print('go SDC-LS')
        time_start_SDC_LS = time.time()
        x_out_SDC_LS,F_out_SDC_LS,k_out_SDC_LS,dca_used_SDC_LS = SDC(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SDC_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SDC_LS-time_start_SDC_LS)
        values.append(F_out_SDC_LS)
        method.append('SDC-LS')
        num_iters.append(k_out_SDC_LS)
        extraps.append('no')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SDC_LS)

        print( '')

        print('go SC-LS')
        time_start_SC_LS = time.time()
        x_out_SC_LS,F_out_SC_LS,k_out_SC_LS,dca_used_SC_LS = SDC(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(0),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SC_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SC_LS-time_start_SC_LS)
        values.append(F_out_SC_LS)
        method.append('SC-LS')
        num_iters.append(k_out_SC_LS)
        extraps.append('no')
        DC_updates.append('no')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SC_LS)

        print(' ')




        subdf = pd.DataFrame(data={'repID':repetitions,'vals':values,'times':times,'method':method,'iters':num_iters,\
                                   'extrapolation':extraps,'dc_updates':DC_updates,'linesarch':ls,'pID':probIDs,'num_used_dc_steps':dca_useful_iters})    

        if fade<0:
            subdf.to_csv('benchmark_methods_concave_LSE_QP_d_'+str(d)+'.csv')
        elif fade>0:
            subdf.to_csv('benchmark_methods_convex_LSE_QP_d_'+str(d)+'.csv')
        else:
            subdf.to_csv('benchmark_methods_neutral_LSE_QP_d_'+str(d)+'.csv')
            
    print(' ')

 
PID #0
 
repID: 0
 
 
go ASDCe
#1 itrs. | Fval = 2.9949e+00 ; div. 3.43e-04 ; L = 1.500e+03 ; #DC iters = 0 ; B_size = 3
#2 itrs. | Fval = 2.5660e+00 ; div. 9.69e-05 ; L = 2.250e+03 ; #DC iters = 0 ; B_size = 5
#3 itrs. | Fval = 2.4059e+00 ; div. 2.00e-05 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 8
#4 itrs. | Fval = 2.3153e+00 ; div. 9.67e-06 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 7
#5 itrs. | Fval = 2.2395e+00 ; div. 5.02e-06 ; L = 2.563e+04 ; #DC iters = 0 ; B_size = 6
#6 itrs. | Fval = 2.1858e+00 ; div. 3.65e-06 ; L = 2.563e+04 ; #DC iters = 0 ; B_size = 3
#7 itrs. | Fval = 2.1374e+00 ; div. 3.36e-06 ; L = 2.563e+04 ; #DC iters = 0 ; B_size = 2
#8 itrs. | Fval = 2.0821e+00 ; div. 3.30e-06 ; L = 2.563e+04 ; #DC iters = 0 ; B_size = 4
#9 itrs. | Fval = 2.0308e+00 ; div. 3.27e-06 ; L = 2.563e+04 ; #DC iters = 0 ; B_size = 2
#10 itrs. | Fval = 1.9848e+00 ; div. 2.78e-06 ; L = 3.844e+04 ; #DC iters = 0 ; B_size = 4
#11 itrs. | Fval = 1.9395e+00 ; div. 2.71e-06 ; L = 3.844e+04 ; #D

#91 itrs. | Fval = -6.7396e+00 ; div. 6.84e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#92 itrs. | Fval = -6.8789e+00 ; div. 7.47e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#93 itrs. | Fval = -7.0175e+00 ; div. 7.99e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 4
#94 itrs. | Fval = -7.1471e+00 ; div. 8.61e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#95 itrs. | Fval = -7.2737e+00 ; div. 6.25e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 5
#96 itrs. | Fval = -7.3830e+00 ; div. 6.35e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#97 itrs. | Fval = -7.4892e+00 ; div. 6.57e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#98 itrs. | Fval = -7.5989e+00 ; div. 6.82e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#99 itrs. | Fval = -7.7240e+00 ; div. 7.07e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 4
#100 itrs. | Fval = -7.8593e+00 ; div. 7.65e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 3
#101 itrs. | Fval = -7.9981e+00 ; div. 8.28e-05 ; L = 1.139e+04 ; #DC iters = 0

#80 itrs. | Fval = -1.5541e+00 ; div. 1.84e-05 ; L = 1.139e+04 ; #DC iters = 0
#90 itrs. | Fval = -2.3039e+00 ; div. 1.27e-05 ; L = 3.844e+04 ; #DC iters = 0
#100 itrs. | Fval = -3.0092e+00 ; div. 8.29e-06 ; L = 5.767e+04 ; #DC iters = 0
#110 itrs. | Fval = -3.5798e+00 ; div. 1.82e-05 ; L = 1.139e+04 ; #DC iters = 0
#120 itrs. | Fval = -4.0610e+00 ; div. 7.74e-06 ; L = 1.709e+04 ; #DC iters = 0
#130 itrs. | Fval = -4.6486e+00 ; div. 4.07e-06 ; L = 5.767e+04 ; #DC iters = 0
#140 itrs. | Fval = -5.2518e+00 ; div. 2.66e-05 ; L = 1.139e+04 ; #DC iters = 0
#150 itrs. | Fval = -6.0457e+00 ; div. 2.96e-05 ; L = 1.139e+04 ; #DC iters = 0
#160 itrs. | Fval = -7.1160e+00 ; div. 4.16e-05 ; L = 1.709e+04 ; #DC iters = 0
#170 itrs. | Fval = -8.0065e+00 ; div. 4.04e-05 ; L = 1.709e+04 ; #DC iters = 0
#180 itrs. | Fval = -9.0455e+00 ; div. 2.53e-05 ; L = 2.563e+04 ; #DC iters = 0
#190 itrs. | Fval = -1.0085e+01 ; div. 4.74e-05 ; L = 1.139e+04 ; #DC iters = 0
#200 itrs. | Fval = -1.1000e+01 ; div. 1.3

#230 itrs. | Fval = -1.4144e+00 ; div. 1.03e-06 ; L = 1.139e+04 ; #DC iters = 0
#240 itrs. | Fval = -1.5866e+00 ; div. 3.24e-07 ; L = 1.709e+04 ; #DC iters = 0
#250 itrs. | Fval = -1.8078e+00 ; div. 1.69e-07 ; L = 2.563e+04 ; #DC iters = 0
#260 itrs. | Fval = -2.0515e+00 ; div. 1.52e-06 ; L = 7.594e+03 ; #DC iters = 0
#270 itrs. | Fval = -2.2650e+00 ; div. 6.09e-08 ; L = 3.844e+04 ; #DC iters = 0
#280 itrs. | Fval = -2.5620e+00 ; div. 9.02e-08 ; L = 8.650e+04 ; #DC iters = 0
#290 itrs. | Fval = -2.7088e+00 ; div. 6.90e-06 ; L = 3.375e+03 ; #DC iters = 0
#300 itrs. | Fval = -2.9630e+00 ; div. 1.71e-07 ; L = 5.767e+04 ; #DC iters = 0
#310 itrs. | Fval = -3.0972e+00 ; div. 3.74e-08 ; L = 8.650e+04 ; #DC iters = 0
#320 itrs. | Fval = -3.2279e+00 ; div. 6.56e-06 ; L = 3.375e+03 ; #DC iters = 0
#330 itrs. | Fval = -3.3342e+00 ; div. 5.78e-07 ; L = 1.139e+04 ; #DC iters = 0
#340 itrs. | Fval = -3.5338e+00 ; div. 6.76e-07 ; L = 1.139e+04 ; #DC iters = 0
#350 itrs. | Fval = -3.6797e+00 ; div. 3

#250 itrs. | Fval = -1.8078e+00 ; div. 1.69e-07 ; L = 2.563e+04 ; #DC iters = 0
#260 itrs. | Fval = -2.0513e+00 ; div. 1.52e-06 ; L = 7.594e+03 ; #DC iters = 0
#270 itrs. | Fval = -2.2643e+00 ; div. 6.39e-08 ; L = 3.844e+04 ; #DC iters = 0
#280 itrs. | Fval = -2.5542e+00 ; div. 5.41e-08 ; L = 1.297e+05 ; #DC iters = 0
#290 itrs. | Fval = -2.6619e+00 ; div. 3.04e-06 ; L = 5.062e+03 ; #DC iters = 0
#300 itrs. | Fval = -2.8360e+00 ; div. 1.38e-06 ; L = 7.594e+03 ; #DC iters = 0
#310 itrs. | Fval = -3.0508e+00 ; div. 2.85e-07 ; L = 1.709e+04 ; #DC iters = 0
#320 itrs. | Fval = -3.2328e+00 ; div. 5.93e-08 ; L = 3.844e+04 ; #DC iters = 0
#330 itrs. | Fval = -3.3996e+00 ; div. 8.99e-09 ; L = 1.297e+05 ; #DC iters = 0
#340 itrs. | Fval = -3.5498e+00 ; div. 1.94e-06 ; L = 7.594e+03 ; #DC iters = 0
#350 itrs. | Fval = -3.7183e+00 ; div. 3.07e-06 ; L = 5.062e+03 ; #DC iters = 0
#360 itrs. | Fval = -3.9090e+00 ; div. 7.06e-06 ; L = 3.375e+03 ; #DC iters = 0
#370 itrs. | Fval = -4.0448e+00 ; div. 1

...

#89 itrs. | Fval = -8.0205e+00 ; div. 8.46e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#90 itrs. | Fval = -8.1646e+00 ; div. 9.10e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#91 itrs. | Fval = -8.3190e+00 ; div. 9.69e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 4
#92 itrs. | Fval = -8.4733e+00 ; div. 7.15e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 4
#93 itrs. | Fval = -8.6144e+00 ; div. 7.32e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#94 itrs. | Fval = -8.7536e+00 ; div. 7.56e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#95 itrs. | Fval = -8.9053e+00 ; div. 8.28e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#96 itrs. | Fval = -9.0784e+00 ; div. 9.79e-05 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 2
#97 itrs. | Fval = -9.2668e+00 ; div. 1.27e-04 ; L = 5.062e+03 ; #DC iters = 0 ; B_size = 2
#98 itrs. | Fval = -9.4711e+00 ; div. 1.79e-04 ; L = 3.375e+03 ; #DC iters = 1 ; B_size = 2
#99 itrs. | Fval = -9.6423e+00 ; div. 1.49e-04 ; L = 5.062e+03 ; #DC iters = 2 ;

#10 itrs. | Fval = 1.7703e+00 ; div. 8.60e-07 ; L = 1.297e+05 ; #DC iters = 0
#20 itrs. | Fval = 1.4548e+00 ; div. 1.43e-06 ; L = 8.650e+04 ; #DC iters = 0
#30 itrs. | Fval = 9.5987e-01 ; div. 3.93e-06 ; L = 5.767e+04 ; #DC iters = 0
#40 itrs. | Fval = 2.5464e-01 ; div. 7.04e-06 ; L = 5.767e+04 ; #DC iters = 0
#50 itrs. | Fval = -5.5126e-01 ; div. 1.20e-05 ; L = 2.563e+04 ; #DC iters = 0
#60 itrs. | Fval = -1.1908e+00 ; div. 8.77e-06 ; L = 1.709e+04 ; #DC iters = 0
#70 itrs. | Fval = -1.8182e+00 ; div. 1.24e-05 ; L = 2.563e+04 ; #DC iters = 0
#80 itrs. | Fval = -2.4128e+00 ; div. 4.10e-06 ; L = 8.650e+04 ; #DC iters = 0
#90 itrs. | Fval = -2.9356e+00 ; div. 1.18e-05 ; L = 2.563e+04 ; #DC iters = 0
#100 itrs. | Fval = -3.8091e+00 ; div. 3.22e-05 ; L = 1.709e+04 ; #DC iters = 0
#110 itrs. | Fval = -4.6731e+00 ; div. 3.36e-05 ; L = 1.139e+04 ; #DC iters = 0
#120 itrs. | Fval = -5.3566e+00 ; div. 8.59e-06 ; L = 3.844e+04 ; #DC iters = 0
#130 itrs. | Fval = -6.0037e+00 ; div. 1.13e-05 ; L =

#590 itrs. | Fval = -9.3876e+00 ; div. 2.53e-07 ; L = 1.709e+04 ; #DC iters = 0
#600 itrs. | Fval = -9.7277e+00 ; div. 2.93e-06 ; L = 5.062e+03 ; #DC iters = 1
#610 itrs. | Fval = -9.9720e+00 ; div. 2.94e-07 ; L = 1.709e+04 ; #DC iters = 1
#620 itrs. | Fval = -1.0300e+01 ; div. 1.29e-06 ; L = 7.594e+03 ; #DC iters = 1
#630 itrs. | Fval = -1.0509e+01 ; div. 2.70e-06 ; L = 5.062e+03 ; #DC iters = 1
#640 itrs. | Fval = -1.0903e+01 ; div. 5.24e-06 ; L = 5.062e+03 ; #DC iters = 1
#650 itrs. | Fval = -1.1177e+01 ; div. 1.28e-05 ; L = 2.250e+03 ; #DC iters = 1
#660 itrs. | Fval = -1.1413e+01 ; div. 2.57e-06 ; L = 5.062e+03 ; #DC iters = 1
#670 itrs. | Fval = -1.1722e+01 ; div. 1.24e-05 ; L = 2.250e+03 ; #DC iters = 1
#680 itrs. | Fval = -1.2075e+01 ; div. 3.82e-06 ; L = 5.062e+03 ; #DC iters = 1
#690 itrs. | Fval = -1.2344e+01 ; div. 3.17e-06 ; L = 5.062e+03 ; #DC iters = 1
#700 itrs. | Fval = -1.2688e+01 ; div. 3.70e-05 ; L = 1.500e+03 ; #DC iters = 1
#710 itrs. | Fval = -1.3058e+01 ; div. 3

#620 itrs. | Fval = -1.0777e+01 ; div. 4.19e-06 ; L = 5.062e+03 ; #DC iters = 0
#630 itrs. | Fval = -1.1055e+01 ; div. 1.31e-05 ; L = 2.250e+03 ; #DC iters = 0
#640 itrs. | Fval = -1.1316e+01 ; div. 5.86e-06 ; L = 3.375e+03 ; #DC iters = 0
#650 itrs. | Fval = -1.1551e+01 ; div. 4.78e-06 ; L = 5.062e+03 ; #DC iters = 0
#660 itrs. | Fval = -1.1823e+01 ; div. 2.74e-06 ; L = 5.062e+03 ; #DC iters = 0
#670 itrs. | Fval = -1.2092e+01 ; div. 5.20e-07 ; L = 1.139e+04 ; #DC iters = 0
#680 itrs. | Fval = -1.2402e+01 ; div. 5.22e-06 ; L = 3.375e+03 ; #DC iters = 0
#690 itrs. | Fval = -1.2754e+01 ; div. 2.98e-06 ; L = 5.062e+03 ; #DC iters = 0
#700 itrs. | Fval = -1.3213e+01 ; div. 3.29e-06 ; L = 7.594e+03 ; #DC iters = 0
#710 itrs. | Fval = -1.3649e+01 ; div. 2.19e-05 ; L = 2.250e+03 ; #DC iters = 0
#720 itrs. | Fval = -1.4030e+01 ; div. 1.80e-06 ; L = 7.594e+03 ; #DC iters = 0
#730 itrs. | Fval = -1.4464e+01 ; div. 1.99e-06 ; L = 7.594e+03 ; #DC iters = 0
#740 itrs. | Fval = -1.4919e+01 ; div. 2

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
sns.set_theme()
sns.relplot(
    data=subdf, x="times", y="vals",
    hue="method", style="dc_updates",
    kind="scatter",col='extrapolation'
)

In [ ]:
for f in list(subdf['extrapolation'].unique()):
    subgroup = subdf[(subdf['extrapolation']==f)]
    sub = ['extrapolation: '+str(f)]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')

In [ ]:
for f in list(subdf['dc_updates'].unique()):
    subgroup = subdf[(subdf['dc_updates']==f)]
    sub = ['DC models: '+str(f)]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')

In [ ]:
for f in list(subdf['method'].unique()):
    subgroup = subdf[(subdf['method']==f)]
    sub = [f]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')